<h1 style="color: #00BFFF;">Python-SQL Connection</h1>


In [17]:
# 📚 Basic libraries
import pandas as pd # data manipulation
import numpy as np # numerical operations
import warnings # warning messages management
# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

# 🌐 New Libraries
from getpass import getpass # get password without showing it
import pymysql # MySQL connection
import requests # HTTP requests
import time # time operations

# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

def create_connection(): # create a connection to a MySQL database
    """
    Create a connection to a MySQL database.
    Prompts the user for the database password and database name.
    
    Returns:
        cnx: pymysql.connections.Connection object
            The connection object to the MySQL database.
    """
    password = getpass("Please, kindly insert your password: ")
    database = input("Please, kindly insert your database name: ")
    cnx = pymysql.connect(user='root', password=password,
                          host='localhost', database=database)
    if cnx.open:
        print("Connection open")
    else:
        print("Connection is not successfully open")
    return cnx

In [18]:
engine = create_connection()

Connection open


<h3 style="color: #00BFFF;">Testing the connection</h3>

In [19]:
query = "SELECT * FROM rental;" 
df = pd.read_sql(query, engine)
df.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [20]:
def rentals_month(month, year):
    query_1 = f"""
    SELECT customer_id, rental_date
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    rentals_df = pd.read_sql(query_1, engine)
    return rentals_df

In [21]:
rentals_may = rentals_month( 5, 2005)
rentals_may

,customer_id,rental_date
0,130,2005-05-24 22:53:30
1,459,2005-05-24 22:54:33
2,408,2005-05-24 23:03:39
3,333,2005-05-24 23:04:41
4,222,2005-05-24 23:05:21
...,...,...
1151,506,2005-05-31 21:36:44
1152,59,2005-05-31 21:42:09
1153,251,2005-05-31 22:17:11
1154,106,2005-05-31 22:37:34


In [22]:
def rental_count_month(rentals_df, month, year):
    rental_counts = rentals_df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_counts

In [23]:
rentals_may_count = rental_count_month(rentals_may, 5, 2005)
rentals_may_count

,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


In [24]:
def compare_rentals(df1, df2):
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('', '_other')).fillna(0)
    combined_df['difference'] = combined_df.iloc[:, 1] - combined_df.iloc[:, 2]
    return combined_df

In [25]:
rentals_june = rentals_month( 6, 2005)
rentals_june

,customer_id,rental_date
0,416,2005-06-14 22:53:33
1,516,2005-06-14 22:55:13
2,239,2005-06-14 23:00:34
3,285,2005-06-14 23:07:08
4,310,2005-06-14 23:09:38
...,...,...
2306,510,2005-06-21 22:10:01
2307,226,2005-06-21 22:13:33
2308,543,2005-06-21 22:19:25
2309,100,2005-06-21 22:43:45


In [26]:
rentals_june_count = rental_count_month(rentals_june, 6, 2005)
rentals_june_count

,customer_id,rentals_06_2005
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
...,...,...
585,595,2
586,596,2
587,597,3
588,598,1


In [27]:
diference = compare_rentals(rentals_may_count,rentals_june_count)
diference

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,-5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,-2.0
3,4,0.0,6.0,-6.0
4,5,3.0,5.0,-2.0
...,...,...,...,...
593,595,1.0,2.0,-1.0
594,596,6.0,2.0,4.0
595,597,2.0,3.0,-1.0
596,598,0.0,1.0,-1.0
